# Ejercicio 7 - Keras tuner

## Carga de imágenes
Cargamos las imagenes del ejercicio anterior:

In [27]:
!pip install -q -U keras-tuner
!pip install argparse

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [28]:
!wget --no-check-certificate \
https://www.dropbox.com/s/sshnskxxolkrq9h/cats_and_dogs_small.zip?dl=0 \
-O /tmp/cats_and_dogs_small.zip

--2022-04-29 10:07:14--  https://www.dropbox.com/s/sshnskxxolkrq9h/cats_and_dogs_small.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/sshnskxxolkrq9h/cats_and_dogs_small.zip [following]
--2022-04-29 10:07:14--  https://www.dropbox.com/s/raw/sshnskxxolkrq9h/cats_and_dogs_small.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc555aae26d88ded30bfae4a50e6.dl.dropboxusercontent.com/cd/0/inline/BkSFpSiWsGr0XknaTS0ay0cm6ShOP7dzo3o1aRtQ7fuulj11IYSnfaJMgFjT2PBFHWOFQXZqjIBvjlDp5XqwP9mDKp4_e2yy5KiITJcg1INay-LUItMg9acc18cnvUx698Tsx0JDUFpEajBLRVgAAe0ZaweZsUiHeY2YkjndRxFmPw/file# [following]
--2022-04-29 10:07:15--  https://uc555aae26d88ded30bfae4a50e6.dl.dropboxusercontent.com/cd/0/inline/BkSFpSiWsGr0XknaTS0ay0cm6Sh

Separamos las imágenes por carpetas:

In [29]:
import os
import zipfile
local_zip = '/tmp/cats_and_dogs_small.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Directorio con las imagenes de training
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directorio con las imagenes de validation
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Directorio con las imagenes de test
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

# Comprobamos el numero total de imagenes en cada directorio
print('total training cat images :', len(os.listdir(train_cats_dir ) ))
print('total training dog images :', len(os.listdir(train_dogs_dir ) ))
print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))
print('total test cat images :', len(os.listdir( test_cats_dir ) ))
print('total test dog images :', len(os.listdir( test_dogs_dir ) ))
print('-------------------------------')


total training cat images : 1000
total training dog images : 1000
total validation cat images : 500
total validation dog images : 500
total test cat images : 500
total test dog images : 500
-------------------------------


## Generación de nuevos datos y entrenamiento

Es importante resaltar que el aumento de datos de imgen generalmente solo se aplica al conjunto de datos de entreneamiento, y no al conjunto de datos de validación o prueba

## Creamos los parametros de configuracion:

In [30]:
import numpy as np
import pandas as pd
import time
# define the path to our output directory
OUTPUT_PATH = "output"
TUNER_NAME = "tuner_test"
# initialize the input shape and number of classes
#INPUT_SHAPE = (150, 150, 3)
#NUM_CLASSES = 2
# define the total number of epochs to train, batch size, and the
# early stopping patience
EPOCHS = 5
BS = 200 # batch size
EARLY_STOPPING_PATIENCE = 1

# ['epochs' 'batch_size' 'optimizer' 'layer_n' 'learning_rate' 'time']
layer_names =['study_dense_units']
df_columns = ['epochs', 'batch_size', 'optimizer']
for name in layer_names:
  df_columns = np.append(df_columns, name)
df_columns = np.append(df_columns, ['learning_rate', 'test_acc'])

df = pd.DataFrame(columns=df_columns)

In [31]:
def update_dataframe(df, df_columns, epochs, batch_size, optimizer, layer_values, learning_rate, test_acc):
  data = [epochs, batch_size, optimizer]
  for value in layer_values:
    data = data + layer_values
  data = data + [learning_rate, test_acc]
  new_data = [tuple(data)]
  new_df = pd.DataFrame(new_data, columns=df_columns)
  return df.append(new_df)

#update_dataframe(pd.DataFrame(), df_columns, 3, 50, 'hyperband', [128], 0.001, 98.2)

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

batch = BS
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                    batch_size=batch,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

test_generator = test_datagen.flow_from_directory(test_dir,
                                                    batch_size=batch,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

                                               

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Implementamos la funcion de plot

In [33]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary package
import matplotlib.pyplot as plt
def save_plot(H, path):
	# plot the training loss and accuracy
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["acc"], label="train_acc")
	plt.plot(H.history["val_acc"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend()
	plt.savefig(path)

## Creando nuestro modelo

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

def build_model(hp):
  #initialize th model along with the input chape and channel
  #dimension
  pre_trained_model = VGG16(input_shape = (150,150,3),
                                include_top = False,
                                weights = 'imagenet')
  
  for layer in pre_trained_model.layers:
    layer.trainable = False

  modelFE = tf.keras.models.Sequential([
      pre_trained_model,
      tf.keras.layers.Flatten(),
      # Creamos una capa con parametros variables
      tf.keras.layers.Dense(
          hp.Int(layer_names[0], min_value=256, max_value=512, step=128),
          activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
      #tf.keras.layers.Dense(256, activation='relu'),
      #tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  '''
  modelFE = tf.keras.models.Sequential()
  modelFE.add(pre_trained_model)
  for layer in layer_names:
    modelFE.add(hp.Int(layer, min_value=128, max_value=256, step=128),activation='relu'),
  modelFE.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  '''
  # Inicializamos el factor de aprendizaje y el optimizador
  #1e-1, 1e-2, 1e-3, 1e-4
  lr = hp.Choice("learning_rate", 
                 values=[1e-2, 1e-3, 1e-4]) # valores de aprendizaje
  opt = Adam(learning_rate=lr) # optimizador

  # Compilamos el modelo
  modelFE.compile(optimizer=opt, 
                loss='binary_crossentropy', 
                metrics=['acc'])
  
  return modelFE


## Implementamos el ajuste de hiperparametros

In [35]:
def test_model(bestHP):
  # build the best model and train it
  print("[INFO] training the best model...")
  model = tuner.hypermodel.build(bestHP)
  history = model.fit(
    train_generator,
    validation_data=validation_generator, 
    validation_steps=validation_steps,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    verbose=2
  )

  # evaluate the network
  test_lost, test_acc= model.evaluate(test_generator)
  print ("Test Accuracy:", test_acc)
  return test_acc

In [36]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping # Nos permite detener un entrenamiento que rinde de forma pobre
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report
import keras_tuner as kt # Paquete que nos permite implementar el ajuste de hiperparametros
import numpy as np
import argparse
import cv2

import numpy as np
import pandas as pd
import time
choices=["hyperband", "random", "bayesian"]
#choices = ["hyperband"]

# initialize an early stopping callback to prevent the model from
# overfitting/spending too much time training with minimal gains
es = EarlyStopping(
	monitor="val_loss",
	patience=EARLY_STOPPING_PATIENCE,
	restore_best_weights=True)

# Tenemos que instanciar el afinador para cada tipo de optimizador
for optimizer in choices:
	# check if we will be using the hyperband tuner
	if optimizer == "hyperband":
		# instantiate the hyperband tuner object
		print("[INFO] instantiating a hyperband tuner object...")
		tuner = kt.Hyperband(
			build_model,
			objective="val_acc",
			max_epochs=EPOCHS,
			factor=3,
			seed=42,
			directory=OUTPUT_PATH,
			project_name=TUNER_NAME,
			overwrite=True
		)
	elif optimizer == "random":
		# instantiate the random search tuner object
		print("[INFO] instantiating a random search tuner object...")
		tuner = kt.RandomSearch(
			build_model,
			objective="val_acc",
			max_trials=10,
			seed=42,
			directory=OUTPUT_PATH,
			project_name=TUNER_NAME,
			overwrite=True
		)
	else:
		# instantiate the bayesian optimization tuner object
		print("[INFO] instantiating a bayesian optimization tuner object...")
		tuner = kt.BayesianOptimization(
			build_model,
			objective="val_acc",
			max_trials=10,
			seed=42,
			directory=OUTPUT_PATH,
			project_name=TUNER_NAME,
			overwrite=True
		)

	# perform the hyperparameter search
	steps_per_epoch = train_generator.n / BS
	validation_steps = validation_generator.n / BS
	print("Steps per epoch: {}".format(steps_per_epoch))
	print("validation_steps: {}".format(validation_steps))
	print("")

	# Realizamos la busqueda de hyperparametros en el afinador instanciado previamente
	print("[INFO] performing hyperparameter search...")
	tuner.search(
		train_generator,
		validation_data=validation_generator,
		batch_size=BS,
		callbacks=[es],
		epochs=EPOCHS
	)

	# grab the best hyperparameters
	bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
	optimal_numbers = []
	for layer in layer_names:
		number = bestHP.get(layer)
		optimal_numbers = optimal_numbers + [number]
		print(f"[INFO] optimal number of filter in {layer}: {number}")
	
	learning_rate = bestHP.get("learning_rate")
	print("[INFO] optimal learning rate: {:.4f}".format(learning_rate))

	# Comprobamos el mejor resultado
	test_acc = test_model(bestHP)
	
	df = update_dataframe(df,df_columns,EPOCHS,BS,optimizer,optimal_numbers,learning_rate, test_acc)
	#print(df)	
	df.to_csv('./csv/estudio.csv')

print(df)

Trial 4 Complete [00h 01m 56s]
val_acc: 0.8799999952316284

Best val_acc So Far: 0.8820000290870667
Total elapsed time: 00h 08m 49s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
512               |512               |study_dense_units
0.01              |0.001             |learning_rate

Epoch 1/5
10/10 [==============================] - 26s 3s/step - loss: 19.4816 - acc: 0.4950 - val_loss: 4.3044 - val_acc: 0.5020


KeyboardInterrupt: ignored

In [ ]:
'''
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

plt.style.use("ggplot")
acc = history.history['acc'] # Obtenemos los valores de training accuracy 
val_acc = history.history[ 'val_acc' ] # Obtenemos los valores de validation accuracy
loss = history.history['loss'] # Obtenemos los valores de training loss
val_loss = history.history['val_loss' ] # Obtenemos los valores de validation loss

epochs = range(1,len(acc)+1,1)
# Creamos una nueva grafica para el parametro Accuracy
plt.plot (epochs, acc, 'r--', label='Training acc' ) # Mostramos training acc
plt.plot (epochs, val_acc, 'b', label='Validation acc') # Mostramos validation acc
plt.title ('Training and validation accuracy') # Titulo de la grafica
plt.ylabel('acc') # ylabel
plt.xlabel('epochs') # xlabel
plt.legend() # Activamos la leyenda
'''